In [3]:
#https://python.langchain.com/v0.2/docs/tutorials/rag/

In [53]:
! pip install langchain
! pip install gradio

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 3.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 6.1 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: websockets
    Found existing installation: websockets 13.1
    Uninstalling websockets-13.1:
      Successfully uninstalled websockets-13.1


In [5]:
! pip install tenacity==8.5.0

Defaulting to user installation because normal site-packages is not writeable


In [6]:
! pip install --quiet --upgrade langchain langchain-community langchain-chroma

In [15]:
! pip install -qU langchain-openai
! pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [3]:
#make sure to replace the api key with your own! i don't wanna be charged for too many calls LMAOO
from dotenv import load_dotenv
import os

load_dotenv(override=True)

langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key

In [37]:
#again, replace with your own open ai api key
#not sure how fast we're gonna use our free calls but we should prob create a local free rag application with ollama if goldhirsh doesn't give us funding soon
#os.environ['OPENAI_API_KEY'] = ""

In [4]:
from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path

file_path='./idea-2024-new.json'
data = json.loads(Path(file_path).read_text(encoding='utf-8'))
loader = JSONLoader(
         file_path=file_path,
         jq_schema=".[].summary",
         text_content=True)

In [5]:
docs = loader.load()

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

874

In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [8]:
retriever = vectorstore.as_retriever()

In [9]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("Which organization helps the homeless?")

'Vision Possible and Food on Foot are two organizations that help the homeless in Los Angeles. Vision Possible focuses on helping the homeless regain their lives and be productive citizens, while Food on Foot provides meals, supplies, and assistance in securing jobs and housing for the homeless.'

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

import gradio as gr
import time

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

green_theme = gr.themes.Base(
    primary_hue=gr.themes.Color(
        c50="#00A168",    # main green 
        c100="#57B485",   # accent green
        c200="#D7ECE0",   # light green background color
        c300="#FFFFFF",   # White background
        c400="#EAE9E9",   # Light grey border
        c500="#000000",   # Black text
        c600="#3A905E",   # Darker green for hover or other accents
        c700="#2A774A",   # Even darker green for deeper accents
        c800="#1A5E36",   # Dark green, deeper accent
        c900="#0A4512",   # Very dark green
        c950="#052A08"    # Deepest green for high contrast
    ),
    font=[gr.themes.GoogleFont('Space Grotesk'), 'ui-sans-serif', 'system-ui', 'sans-serif']
).set(
    body_background_fill='#00A168',               # green background
    body_text_color='#000000',                    # Black text for readability
    background_fill_primary='#FFFFFF',            # Light green for user message background
    background_fill_secondary='#FFFFFF',          # Set input textbox color to light grey
    border_color_accent='#57B485',                # Green border accent
    border_color_accent_subdued='#EAE9E9',        # Light grey for less prominent borders
    color_accent='#57B485',                       # Green accent for primary elements
    color_accent_soft='#D7ECE0',                  # Softer green accent
    checkbox_background_color='#FFFFFF',          # Light grey for checkbox backgrounds
    button_primary_background_fill='#57B485',     # Green background for primary buttons
    button_primary_background_fill_hover='#3A905E', # Darker green on hover
    button_secondary_background_fill='#D7ECE0',   # Light green background for secondary buttons
    button_secondary_text_color='#000000'         # Black text for secondary buttons
)



# integrate gradio with RAG logic
def message_and_history(input, history):
    history = history or []
    history.append({"role": "user", "content": input})

    response = rag_chain.invoke({"input": input})
    answer = response["answer"]
    chatbot_message = {"role": "assistant", "content": ""}
    history.append(chatbot_message)

    for character in answer:
        chatbot_message["content"] += character
        time.sleep(0.05)
        yield history, history

# set to light mode
js_func = """
function refresh() {
    const url = new URL(window.location);

    if (url.searchParams.get('__theme') !== 'light') {
        url.searchParams.set('__theme', 'light');
        window.location.href = url.href;
    }
}
"""

# setup gradio interface
with gr.Blocks(theme=green_theme, js=js_func) as block:

    gr.Markdown("<h1><center>Chat with Sunny</center></h1>", elem_classes="custom-header")
    chatbot = gr.Chatbot(type="messages")
    state = gr.State([])
    
    with gr.Row():
        message = gr.Textbox(placeholder="Type a message", label="", elem_classes="custom-textbox", scale=3)
        send_button = gr.Button("", elem_classes="send-button", scale=1)

    send_button.click(message_and_history, inputs=[message, state], outputs=[chatbot, state])


block.css = """""
    .custom-header h1 {
    color: #FFFFFF; /* Change this to the desired color */
    text-align: center;
    }
    .custom-textbox input {
        background-color: #EAE9E9 !important;
    }
    .send-button {
        background: none;
        border: none;
        font-size: 20px;
        padding: 0 10px;
        cursor: pointer;
    }
    .send-button:after {
        content: '\\27A4'; /* Unicode for paper plane icon, or use your preferred icon */
        color: #57B485;
    }
    .send-button:hover:after {
        color: #3A905E;
    }
    """
# launch interface
block.launch(debug=True, share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://f65ce4d2b596d1e9a4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://f65ce4d2b596d1e9a4.gradio.live


In [52]:
for document in response["context"]:
    print(document)
    print()

page_content='Homelessness is not an individual issue. It is a community issue. Our organization, Vision Possible, aims to conduct a  vision to help the homeless regain their lives and be productive citizens like you and me.' metadata={'seq_num': 266, 'source': '/Users/5chan/Desktop/LA2050chatbot/notebooks/idea-2024-new.json', 'start_index': 0}

page_content='This grant will go towards three separate projects/programs that have the same targeted audience – the homeless. The first project is to help reunify homeless people with their families/friends (if they so choose) through the creation of a homeless directory that would be available for the public to use, secondly, raise awareness about the daily struggles the homeless face in LA, and lastly, to promote homeless organizations with free ads.' metadata={'seq_num': 709, 'source': '/Users/5chan/Desktop/LA2050chatbot/notebooks/idea-2024-new.json', 'start_index': 0}

page_content='Food on Foot works to alleviate homelessness in Los Angel

In [52]:
#TO DO: add message history
#https://python.langchain.com/docs/how_to/message_history/

In [53]:
#TO DO: local rag application (so we can do more in depth testing without making too many open ai api calls) - using ollama?
#https://python.langchain.com/v0.2/docs/tutorials/local_rag/